### CS4423 - Networks
Angela Carnevale  
School of Mathematical and Statistical Sciences  
University of Galway

#### 2. Tree and Graph Traversal

# Week 4, lecture 2: Graph Traversal. Shortest Paths and distance. 

## And  a note on Graph Isomorphism


In [ ]:
import networkx as nx
import numpy as np
opts={"with_labels":True, "node_color":'y'}

## Graph Traversal

* A natural problem arising in many practical applications is the following: Given a pair of nodes $x, y$, find one or all the paths from $x$ to $y$ with the **fewest number of edges** possible.

* This is a more complex measure on a network than, say, the degree
of a node. And we will need a more complex procedure, that is, an
algorithm, in order to solve such problems systematically.

Let's start with a proper definition.

**Definition.** Let $G = (X, E)$ be a simple graph and let
$x, y \in X$.  Let $P(x, y)$ be the set of all paths from $x$ to $y$. Then:

* The __distance__ $d(x, y)$ from $x$ to $y$ is
$$d(x, y) = \min \{ l(p) : p \in P(x, y) \},$$
the shortest possible length of a path from $x$ to $y$, and a __shortest path__ from $x$ to $y$ is a path $p \in P(x, y)$ of length $l(p) = d(x, y)$.

* The __diameter__ $\mathrm{diam}(G)$ of the network $G$ is the length of the longest shortest path between any two nodes,
$$\mathrm{diam}(G) = \max \{ d(x, y) : x, y \in X \}.$$

### Breadth First Search for Distance

We now consider the following problem: Given a node $x \in X$ in a graph $G$, what
are the distances $d(x, y)$ for all nodes $y \in X$?  

* BFS provides a systematic
procedure for finding these distances, and the shortest paths through
which they are realized. 

* We will start by describing how BFS works for **graph** traversal.

In order to describe the algorithm step by step, let's call a node $y$
a __neighbor__ (or friend) of node $x$, if $\{x, y\}$ is an
edge, and let's denote by
$$N(x) = \{ y \in X : \{x, y\} \in E \}$$
the set of all neighbors of node $x$. 

The algorithm works through the
network **layer by layer**, starting with the given vertex $x$ at layer
$0$ and all its friends at layer $1$. It then finds the friends of the
friends at layer $2$, and so on, until every node that can be reached
from $x$ by a path has been recorded, taking care that **no node gets
recorded twice**.  

We'll exploit the fact that the layer of a node then corresponds to its distance
from the given node $x$.

In practice, for simple graph traversal, the layers do not
need to be made explicit. 

In [ ]:
!cat data/bfs.adj

In [ ]:
G = nx.read_adjlist("data/bfs.adj")

In [ ]:
nx.draw(G, **opts)

In [ ]:
for x in G: 
    G.nodes[x]['seen'] = False

In [ ]:
G.nodes['A']

In [ ]:
Q = []

In [ ]:
Q.append('A')
G.nodes['A']['seen'] = True
Q

In [ ]:
list(G.neighbors('A'))

In [ ]:
for y in G.neighbors('A'): 
    Q.append(y)
    G.nodes[y]['seen'] = True
    
## for now we skip checking whether the neighbours have been 'seen' or not, 
## but we'll need to add this check to the generic step of the algorithm

In [ ]:
Q

In [ ]:
node = 'B'
for y in G.neighbors(node):
    if not G.nodes[y]['seen']:
        Q.append(y)
        G.nodes[y]['seen'] = True
Q

In [ ]:
node = 'C'
for y in G.neighbors(node):
    if not G.nodes[y]['seen']:
        Q.append(y)
        G.nodes[y]['seen'] = True
Q

... and so on, until there are no more nodes to be processed.

In [ ]:
# 1. initialize
for x in G: 
    G.nodes[x]['seen'] = False

G.nodes['A']['seen'] = True
Q = ['A']

# 2. loop
for node in Q:
    for y in G.neighbors(node):
        if not G.nodes[y]['seen']:
            Q.append(y)
            G.nodes[y]['seen'] = True

# 3. output result
Q

When this process is formulated as an algorithm, we use an explicit __queue__
(a first-in first-out buffer) to keep track of the node
whose neighbors are currently under consideration.
A queue is an array of values that comes with two basic operations:
* one can __push__ a value to the end of the queue, and
* one can __pop__ a value off the top of the queue (provided
the queue is not empty).

It can be shown that this version of the algorithm
in the common case of a [sparse network](https://en.wikipedia.org/wiki/Sparse_network)
has complexity $O(n)$, which is as good as one could hope for.

**Breadth First Search for Distance.**
Given a simple graph
$G = (X, E)$ and a vertex $x \in X$,
determine $d(x, y)$ for all nodes $y \in X$.

1. [Initialize.]  Suppose that $X = \{x_0, x_1, \ldots, x_{n-1}\}$
and that $x = x_j$.  Set $d_i \gets \perp$ (undefined) for $i = 0, \dots, n{-}1$.
Set $d_j \gets 0$ and initialize a queue $Q \gets (x_j)$.

2. [Loop.] While $Q \neq \emptyset$:
   * pop node $x_k$ off $Q$
   * for each neighbor $x_l$ of $x_k$ with $d_l = \perp$:
       * push $x_l$ onto $Q$ and set $d_l \gets d_k + 1$.

3. [Stop.] Return the array $(d_0, \dots, d_{n-1})$.

Note how in this version of BFS, in contrast to the simple version, a node is visited (setting its $d$ attribute)
immediately when it is pushed onto $Q$, rather than later when it pops off $Q$. 

In [ ]:
from queue import Queue

In [ ]:
for x in G: 
    G.nodes[x]['d'] = None # undefined

x = 'B'             #starting BFS at vertex B
G.nodes[x]['d'] = 0 # and setting its distance to 0
Q = Queue()
Q.put(x)

In [ ]:
list(Q.queue)

In [ ]:
while not Q.empty():
    x = Q.get()
    for y in G.neighbors(x):
        if G.nodes[y]['d'] is None: # checking if the distance is undefined
            G.nodes[y]['d'] = G.nodes[x]['d'] + 1 # if so, define using previous
            Q.put(y)
    print(x, ": ", list(Q.queue))

In [ ]:
print([G.nodes[x]['d'] for x in G])

In [ ]:
print(list(G.nodes.data('d')))

## Variants. 

BFS is an extremely versatile algorithm, which applies in many different
situations and can be adapted to produce additional information
on a network.

For example, BFS run on a node $x$ in a network $G = (X, E)$
determines the __connected component__ of $x$ in $G$
(as the set of all nodes that get a distance value assigned). 

With little more work (and an additional array) BFS can produce
a __spanning tree__ (or __shortest path tree__).
Here, whenever node $x_l$ is pushed onto $Q$, it is assigned
the current node $x_k$ (in the additional array)
as its predecessor on a shortest path from $x_j$ to $x_l$.
The subgraph of the network consisting of these edges is a tree.
As a tree, it has exactly one path between the given node $x$
and any of its
vertices $y$ and, by construction, this path is a shortest path
between $x$ and $y$.

In [ ]:
for x in G: 
    G.nodes[x]['d'] = None # set all vertices to undefined

x = 'A' # start with vertex A
G.nodes[x]['d'] = 0    # set its distance to 0
Q = Queue() # initialise a queue Q
Q.put(x)  # push x in Q

for e in G.edges:  
    G.edges[e]['seen'] = False # set all edges to unseen

In [ ]:
while not Q.empty(): 
    x = Q.get() # pop a vertex from the queue
    for y in G.neighbors(x):
        if not G.nodes[y]['d']: # undefined? 
            G.nodes[y]['d'] = G.nodes[x]['d'] + 1 # set distance
            Q.put(y) # push in queue
            G.edges[x, y]['seen'] = True # set relevant edge to seen
    print(x, ": ", list(Q.queue))

In [ ]:
sub = [e for e in G.edges if G.edges[e]['seen']]
# subset of edges 'seen' while visiting the graph

In [ ]:
nx.draw(G.edge_subgraph(sub), **opts)

Or, one could highlight the spanning tree inside the graph by using, say,
red as color for the spanning edges (and blue for the rest).

In [ ]:
colors = ['red' if G.edges[e]['seen'] else 'blue' for e in G.edges]
nx.draw(G, edge_color = colors, with_labels = True, width=2.0)

* Of course, in order to find distances, or shortest paths
between **all pairs** of nodes $x$ and $y$ in a network, one can
perform BFS for each of the vertices $x \in X$ in turn.

* In your next assignment, you will see more in detail an implementation of BFS aimed at constructing a spanning tree.

* The algorithm and its variants also works on directed networks,
but the results then will have to be interpreted in the context of
directed networks.

More about BFS can be found in [Newman, Section 10.3].

#### 2. Tree and Graph Traversal

## A note on Graph Isomorphism and Symmetries

* Two graphs $G = (X, E)$ and $H = (Y, F)$ are said to be **isomorphic** if there
is an edge-preserving bijection between their vertex sets $X$ and $Y$.

* [Deciding graph isomorphism](https://en.wikipedia.org/wiki/Graph_isomorphism_problem)
is computationally hard.

* An isomorphism of a graph $G$ with itself is called an **automorphism**,
or a **symmetry** of $G$.

* Symmetries, or the lack thereof, are interesting properties of networks.

* For instance, in random selections, like the random trees on $n$ vertices, it turns out that **more symmetric species are less frequently picked**.

Morally, this is because graph symmetries are properties of an isomorphism class, rather than a specific graph or network. Let's gather an intuition for this by looking at (small) trees.

## Trees on 4 vertices


![4-trees](images/t4.png)

According to Cayley's formula, there are indeed $4^{4-2} = 16$ **labelled** trees on $n = 4$ vertices.  But overall, we only see $2$
distinct structures: 
* a **path graph** of length $3$, and 
* a **star graph** with $3$ spikes.

These structures are known as **unlabelled trees**
(as opposed to a **labelled tree**, where each node corresponds to
a specific element of $\{0, \dots, n{-}1\}$).

In [ ]:
n = 4
T = nx.random_tree(n)
nx.draw(T)

As a random graph, the path graph occurs far more often than
the star graph.  Is there something wrong with the assumption of uniform
distribution?

No, there isn't.  It's just that the line **shape** appears more often than the
star **shape** in the full list of all **labelled** graphs on 4 points. How often a **shape** appears is a function of how many symmetries (=how many automorphisms) it has.

Morally, fewer symmetries means that a **shape** will occur more often.

##  Code Corner

`python`

* `[].pop` [[doc]](https://docs.python.org/2/tutorial/datastructures.html)

* `[].extend`  [[doc]](https://docs.python.org/2/tutorial/datastructures.html)

### `queue`

* `Queue`: [[doc]](https://docs.python.org/3/library/queue.html)

### `networkx`

* `neighbors`: [[doc]](https://networkx.org/documentation/stable/reference/classes/generated/networkx.Graph.neighbors.html)


* `remove_node`: [[doc]](https://networkx.github.io/documentation/stable/reference/classes/generated/networkx.Graph.remove_node.html)


* `edge_subgraph`: [[doc]](https://networkx.github.io/documentation/stable/reference/classes/generated/networkx.Graph.edge_subgraph.html)

## Exercises.

1. Compute the distances $d(x, y)$ for all vertices $x$ and $y$ in the above graph `G`. 
1. Hence determine the diameter $\mathrm{diam}(G)$.
1. Compute a matrix $D= (d_{ij})$ with entries
$$
d_{ij} = d(i, j),
$$
the distance between nodes $i$ and $j$ in $H$.